In [68]:
import pandas as pd
import numpy as np
from unidecode import unidecode
import string

## Strategies to try out
*Say we have a budget of $10 per game*

### Baselines :
- Random
- Bet $1 on each player
- Bet $2 on each player on home team

### With model :
- Stake formula
- Optimization -- balance expected returns and variance
- Weight by jumpball probability

In [42]:
def normalize_name(x) :

    for suffix in [' Jr.', ' Sr.', ' III', ' II', ' IV', ' Jr', ' Sr'] :
        x = x.replace(suffix, '')
    x = x.translate(str.maketrans('', '', string.punctuation))

    return unidecode(x).lower()

In [50]:
odds = pd.read_csv('../data/odds_first_basket.csv')
odds = odds[odds['bookmaker'] == 'fanduel']
first_basket = pd.read_csv('../data/first_basket_2025.csv')
metadata = pd.read_csv('../data/player_metadata.csv')

In [ ]:
odds = odds.merge(
    first_basket[['game_id', 'first_basket']],
    on = 'game_id',
    how = 'left'
)

odds = odds.merge(
    metadata[['player_id', 'name']],
    left_on = 'first_basket', right_on = 'player_id',
    how = 'left'
)

odds['name_x'] = odds['name_x'].apply(normalize_name)
odds['name_y'] = odds['name_y'].apply(normalize_name)

odds['first_basket_scorer'] = (odds['name_x'] == odds['name_y']).astype(int)

In [75]:
def evaluate_strategy(odds, strategy = 'random') :

    if strategy == 'random' :

        odds['stake'] = np.random.uniform(0, 2, (odds.shape[0],))

    odds['return'] = odds['stake'] * (odds['price'] * odds['first_basket_scorer'] - 1)

    return odds

In [77]:
evaluate_strategy(odds).sort_values('return', ascending = False)

,name_x,price,bookmaker,update_time,game_id,event_id,insert_timestamp_utc,first_basket,player_id,name_y,stake,first_basket_scorer,return
9,goga bitadze,11.0,fanduel,2024-12-01T03:14:37Z,202412010BRK,63d09cdb1e59c1435acde1892c74c9d0,2024-12-01 03:15:06.181798+00:00,bitadgo01,bitadgo01,goga bitadze,1.451095,1,14.510954
31,michael porter,8.0,fanduel,2024-12-04T02:45:15Z,202412030DEN,eaf04a3044de875dc6c83132d04ae9f6,2024-12-04 02:45:16.563756+00:00,portemi01,portemi01,michael porter,1.487421,1,10.411950
39,alperen sengun,6.0,fanduel,2024-12-04T02:44:25Z,202412030SAC,30f75d85c74e0811f06c0fbe86044f54,2024-12-04 02:45:16.619025+00:00,sengual01,sengual01,alperen sengun,1.816485,1,9.082425
10,franz wagner,6.0,fanduel,2024-12-04T00:16:13Z,202412030NYK,823e03f1e150f589e94392112743e204,2024-12-04 00:16:23.639726+00:00,wagnefr01,wagnefr01,franz wagner,0.925326,1,4.626631
63,norman powell,8.0,fanduel,2024-12-04T03:29:44Z,202412030LAC,e707fb0dc8825d51e20056a4bcaa4fa8,2024-12-04 03:29:45.059222+00:00,powelno01,powelno01,norman powell,0.607937,1,4.255556
...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,jerami grant,8.5,fanduel,2024-12-04T03:29:44Z,202412030LAC,e707fb0dc8825d51e20056a4bcaa4fa8,2024-12-04 03:29:45.059222+00:00,powelno01,powelno01,norman powell,1.851844,0,-1.851844
7,wendell carter,10.0,fanduel,2024-12-01T03:14:37Z,202412010BRK,63d09cdb1e59c1435acde1892c74c9d0,2024-12-01 03:15:06.181798+00:00,bitadgo01,bitadgo01,goga bitadze,1.860441,0,-1.860441
13,jalen suggs,7.5,fanduel,2024-12-04T00:16:13Z,202412030NYK,823e03f1e150f589e94392112743e204,2024-12-04 00:16:23.639726+00:00,wagnefr01,wagnefr01,franz wagner,1.867109,0,-1.867109
12,jalen brunson,6.0,fanduel,2024-12-04T00:16:13Z,202412030NYK,823e03f1e150f589e94392112743e204,2024-12-04 00:16:23.639726+00:00,wagnefr01,wagnefr01,franz wagner,1.908054,0,-1.908054
